<img src="img/Brilliant_logo.png" width="20%">

### John Mahoney's Teaching Demo

mohnjahoney@gmail.com - mohnjahoney@github.io

In [1]:
# Imports
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import bqplot as bq
import bqplot.pyplot as plt
import numpy as np
import pandas as pd
import copy
import os
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML, Markdown

# Simpson's Paradox

<hr style="height:6px">

Imagine you went to the grocery store and put 3 apples and 4 oranges in your shopping bag.
When you got to the register, you looked in the bag and found only 6 pieces of fruit - that would be strange, right?
Well, what if you looked inside and found **negative 6** pieces of fruit!?

That's a bit like how it feels to encounter **Simpson's Paradox** - our topic for this lesson.

<div class="alert alert-info" role="alert">
    <h2> Simpson's Paradox in a nutshell: </h2>
    <p style="font-size:1.5em"> Combining subsets of data can result in counter-intuitive reversals of trends. </p>
    
<!--        <p class="lead"> Each of two data subsets displays the same trend, yet the whole dataset displays the opposite trend. </p> -->
</div>

In this lesson, we'll explore three curious scenarios, each of which creates a version of this paradox.

Our goals in this lesson are to:
- notice how our intuition may be wrong even in a simple situation,
- learn why our intuition fails, so that these examples seem less paradoxical,
- become aware of the different flavors of this paradox,
- understand how to act when faced with this paradox in real life.

Ready? Let's dig in!.

# EXAMPLE 1: The Shape of Fish.

<hr style="height:6px">

Here we see fish of many different sizes.
We can divide them into a red group and a blue group.
Notice that the red fish are generally "tall" while the blue fish are generally "long".

In [2]:
# Create fish data
fish = pd.DataFrame({'color':['red', 'red', 'red', 'blue', 'blue', 'blue'], 
                     'length':[0.5, 1.0, 1.5, 2.0, 3.0, 4.0], 
                     'height':[2.5, 3.5, 4.5, 1.0, 1.5, 2.0]})

# We don't want the image xspan to be as big as the length coordinate - otherwise the fish will overlap too much.
x_scale = 0.5
y_scale = 0.5

fish['xspan'] = x_scale * fish['length']
fish['yspan'] = y_scale * fish['height']

# We want the fish to start out somewhere "random".
fish['init_x'] = [4.2, 1.5, 3.9, 0.5, 3.1, 1.3]
fish['init_y'] = [0.6, 3.8, 4.4, 4.5, 2.3, 1]

#fish

In [3]:
def show_and_scale_fish(color, xs, ys):
    
    image_pathA = os.path.abspath('./img/fish_red.png')
    image_pathB = os.path.abspath('./img/fish_blue.png')
    
    if color == 'red':
        im = plt.imshow(image=image_pathA, format='filename')
    elif color == 'blue':
        im = plt.imshow(image=image_pathB, format='filename')
    
    # Scale
    im.x = xs
    im.y = ys
    
    return im

def show_and_scale_fish_from_df(row, init=False):
    
    color = row['color']
    
    if init:
        x = row['init_x']
        y = row['init_y']
    else:
        x = row['length']
        y = row['height']

    # NOTE: The extent of the image does not depend on whether it is in "initial" or "normal" position.
    dx = row['xspan']
    dy = row['yspan']
    
    xs = [x - dx/2, x + dx/2]
    ys = [y - dy/2, y + dy/2]
    
    im = show_and_scale_fish(color, xs, ys)
    return im

In [4]:
# Make initial "ocean" plot.

fig_ocean = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':20, 'left':20, 'right':20})

fig_ocean.layout.width = '100%'
fig_ocean.layout.height = '400px'
fig_ocean.layout.border = 'black solid 1px'

# Add fish images
for i in range(len(fish)):
    show_and_scale_fish_from_df(fish.loc[i], init=True)

ocean_xs = np.linspace(0, 5, 400)
ocean_ys = -0.4 * np.abs(np.sin(4*ocean_xs)) + 6.0

ocean_line = plt.plot(ocean_xs, ocean_ys)
    
plt.xlim(0, 6)
plt.ylim(0, 6)

fig_ocean.axes[0].visible = False
fig_ocean.axes[1].visible = False

box_ocean = widgets.HBox([fig_ocean])

box_ocean.layout.width = '400px'
box_ocean.layout.border = None

box_ocean

## VISUALIZE THE DATA

After measuring the dimensions of each fish, we can organize each group in a graph according to their length and height.

In the graphs below, we see strong trends.
First focus on the red fish; As the fish get longer, they also get taller.
Now focus on the blue fish: We see a similar trend - as the fish get longer, they get taller too.

Given two quantities $A$ and $B$, 
when $A$ and $B$ tend to increase together, we say that $A$ and $B$ are **positively correlated**.
When $A$ and $B$ tend to change in opposite directions, we say that $A$ and $B$ are **negatively correlated**.

We just noticed that length and height increase together for the red fish and for the blue fish.
This means that for each group, the length and height are positively correlated.

We can visualize these correlations by drawing a line through each group of fish.
(Try the regression button.)
The slope of each line tells us quantitatively how much the fish height will change for a given change in fish length.

$$\textrm{slope} = \frac{\Delta y}{\Delta x} = \frac{\textrm{difference in fish height}}{\text{difference in fish length}}$$

In [5]:
# Plot the fish in their correct positions on two plots - one red, one blue.

reg_xs = np.linspace(0, 5, 2).reshape(-1, 1)

# Red fish
figA = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

figA.layout.width = '100%'
figA.layout.height = '400px'
figA.layout.border = 'black solid 1px'

# Add fish images
for i in range(len(fish)):
    if fish.loc[i]['color'] == 'red':
        show_and_scale_fish_from_df(fish.loc[i], init=False)

scatterA = plt.scatter(x=fish[fish['color']=='red']['length'], y=fish[fish['color']=='red']['height'], 
                      colors=['Red'])

regA = LinearRegression().fit(fish[['length']][fish['color']=='red'],
                              fish[['height']][fish['color']=='red'])

reg_ysA = regA.predict(reg_xs)
line_regA = plt.plot(x=reg_xs, y=reg_ysA, colors=['Pink'], stroke_width=0)

plt.xlabel('length')
plt.ylabel('height')

plt.xlim(0, 6)
plt.ylim(0, 6)

# Blue fish

figB = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

figB.layout.width = '100%'
figB.layout.height = '400px'
figB.layout.border = 'black solid 1px'

# Add fish images
for i in range(len(fish)):
    if fish.loc[i]['color'] == 'blue':
        show_and_scale_fish_from_df(fish.loc[i], init=False)

scatterB = plt.scatter(x=fish[fish['color']=='blue']['length'], y=fish[fish['color']=='blue']['height'], 
              colors=['Blue'])

regB = LinearRegression().fit(fish[['length']][fish['color']=='blue'],
                              fish[['height']][fish['color']=='blue'])

reg_ysB = regB.predict(reg_xs)
line_regB = plt.plot(x=reg_xs, y=reg_ysB, colors=['lightblue'], stroke_width=0)

plt.xlabel('length')
plt.ylabel('height')

plt.xlim(0, 6)
plt.ylim(0, 6)

# Regression
regression_button = widgets.ToggleButton(value=False, description='Show regression lines')

def on_regression_button(change, line_regA, line_regB):
    
    if change['new'] == True:
        # Show regression
        line_regA.set_trait('stroke_width', 4)
        line_regB.set_trait('stroke_width', 4)
    else:
        # Don't show 
        line_regA.set_trait('stroke_width', 0)
        line_regB.set_trait('stroke_width', 0)
        
regression_button.observe(lambda change:on_regression_button(change, line_regA, line_regB), 'value')

# Container
box_red_blue_figs = widgets.HBox([figA, figB])
box_red_blue = widgets.VBox([box_red_blue_figs, regression_button])

box_red_blue.layout.width = '800px'
box_red_blue.layout.border = 'black solid 1px'

box_red_blue

## QUICK QUIZ:

<div class="alert alert-success" role="alert">
</div>

- Find the blue fish that is 4 feet long. How tall is it?
(answer: 2 feet)

- Imagine you find a red fish that is 2 feet long. How tall do you expect it to be?
(answer: 5.5 feet)

- Blue fish: When the length increases by two feet, the height (increases / decreases) by ___ feet.
(answer: increases, 2/3)

- Think! Consider the number of fishermen ($A$) at a pier and the number of fish ($B$) in the water below. Would you expect that $A$ and $B$ are positively or negatively correlated? Can you argue both sides?

<!-- <div class="alert alert-info" role="alert">
</div> -->

## ...HERE COMES THE PARADOX...

We've seen that, within the group of red fish, longer fish are taller.
The same is true for blue fish.

<p style="font-size:1.5em"> <it>Certainly</it> if we were to consider all fish (red and blue) together, we would continue to find that longer fish are taller. Right?... </p>

Let's put all of the fish in the same graph and see what happens! (Try the aggregation button)

In [6]:
# Aggregate the two groups of fish. Draw an aggregate regression line.

reg_xs = np.linspace(0, 5, 2).reshape(-1, 1)

# All fish
figAB = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':40, 'left':40, 'right':20})

figAB.layout.width = '100%'
figAB.layout.height = '400px'
figAB.layout.border = 'black solid 1px'

# Add fish images
for i in range(len(fish)):
    show_and_scale_fish_from_df(fish.loc[i], init=False)

# dots
scatterA2 = plt.scatter(x=fish[fish['color']=='red']['length'], y=fish[fish['color']=='red']['height'], 
                      colors=['Red'])
scatterB2 = plt.scatter(x=fish[fish['color']=='blue']['length'], y=fish[fish['color']=='blue']['height'], 
                      colors=['Blue'])

# Regressions
regA = LinearRegression().fit(fish[['length']][fish['color']=='red'],
                              fish[['height']][fish['color']=='red'])

reg_ysA = regA.predict(reg_xs)
line_regA2 = plt.plot(x=reg_xs, y=reg_ysA, colors=['Pink'], stroke_width=4)

regB = LinearRegression().fit(fish[['length']][fish['color']=='blue'],
                              fish[['height']][fish['color']=='blue'])

reg_ysB = regB.predict(reg_xs)
line_regB2 = plt.plot(x=reg_xs, y=reg_ysB, colors=['lightblue'], stroke_width=4)

regAB = LinearRegression().fit(fish[['length']],
                              fish[['height']])

reg_ysAB = regAB.predict(reg_xs)
line_regAB2 = plt.plot(x=reg_xs, y=reg_ysAB, colors=['lightgreen'], stroke_width=0)

plt.xlim(0, 6)
plt.ylim(0, 6)


# Aggregate + regression
fish_agg_button = widgets.ToggleButton(value=False, description='Aggregate groups')

# TODO: This is an ugly hack, but it's OK for today.
def on_fish_agg_button(change):
    if change['new'] == True:
        line_regAB2.set_trait('stroke_width', 4)
        line_regA2.set_trait('stroke_width', 0)
        line_regB2.set_trait('stroke_width', 0)
        scatterA2.set_trait('colors', ['green'])
        scatterB2.set_trait('colors', ['green'])
        for i in range(6):
            figAB.marks[i].x = figAB.marks[i].x * 100
    else:
        line_regAB2.set_trait('stroke_width', 0)
        line_regA2.set_trait('stroke_width', 4)
        line_regB2.set_trait('stroke_width', 4)
        scatterA2.set_trait('colors', ['red'])
        scatterB2.set_trait('colors', ['blue'])
        for i in range(6):
            figAB.marks[i].x = figAB.marks[i].x / 100
            
fish_agg_button.observe(on_fish_agg_button, 'value')


# Container
box_red_AND_blue = widgets.VBox([figAB, fish_agg_button])

box_red_AND_blue.layout.width = '400px'
box_red_AND_blue.layout.border = 'black solid 1px'

box_red_AND_blue

Surprisingly, when we aggregate the subgroups, the trend is reversed!
The length and height are now **negatively correlated**.

Note: There is no straight line that we can draw through the set of all fish.
So we draw a line that is "the best fit".
We use the very standard **least-squares regression line** [link](https://en.wikipedia.org/wiki/Least_squares).

<div class="alert alert-info" role="alert">
<!--     <h2> Take-home Message </h2> -->
    <p style="font-size:1.5em"> 
<!-- For each subgroup, longer fish are generally taller. However, when we look at all fish together, longer fish are generally shorter. -->
<!--     <br>
    <br> -->
    Two positive trends can combine to create a negative trend.</p>
</div>

## INTERACTIVE CHALLENGE!

We just discussed an example with two subgroups (of fish).
Let's stretch your understanding a bit.
Can you create a Simpson's Paradox with **three** subgroups?
Move the shapes (circles, squares, and triangles) so that the correlation is **negative within each group**, yet the **overall trend is positive**.

In [7]:
# Play the game!

xmax = 10
ymax = 9

xs = np.array([1.0, 4.0, 6.0, 4, 5, 8, 3, 5, 9])
ys = np.array([8.0, 6.0, 9.0, 2, 1, 4, 1, 4, 3])

circle_inds = [0, 1, 2]
square_inds = [3, 4, 5]
triangle_inds = [6, 7, 8]

fig_challenge1 = plt.figure(fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
# min_aspect_ratio=1.0, max_aspect_ratio=1.01, 

fig_challenge1.layout.width = '100%'
fig_challenge1.layout.height = '300px'
fig_challenge1.layout.border = 'black solid 1px'

reg0 = LinearRegression().fit(xs[circle_inds].reshape(-1, 1), ys[circle_inds])
reg1 = LinearRegression().fit(xs[square_inds].reshape(-1, 1), ys[square_inds])
reg2 = LinearRegression().fit(xs[triangle_inds].reshape(-1, 1), ys[triangle_inds])
reg012 = LinearRegression().fit(xs.reshape(-1, 1), ys)

reg_xs = np.linspace(0, 10, 2)

reg_ys0 = reg0.predict(reg_xs.reshape(-1, 1))
reg_ys1 = reg1.predict(reg_xs.reshape(-1, 1))
reg_ys2 = reg2.predict(reg_xs.reshape(-1, 1))
reg_ys012 = reg012.predict(reg_xs.reshape(-1, 1))

line_reg0 = plt.plot(reg_xs, reg_ys0, colors=['red'], stroke_width=3)
line_reg1 = plt.plot(reg_xs, reg_ys1, colors=['blue'], stroke_width=3)
line_reg2 = plt.plot(reg_xs, reg_ys2, colors=['green'], stroke_width=3)
line_reg012 = plt.plot(reg_xs, reg_ys012, colors=['black'], stroke_width=6, line_style='dashed')

scatter0 = plt.scatter(xs[circle_inds], ys[circle_inds], default_size=600, colors=['white'], 
                       stroke='red', stroke_width=4, marker='circle', enable_move=True)
scatter1 = plt.scatter(xs[square_inds], ys[square_inds], default_size=600, colors=['white'], 
                       stroke='blue', stroke_width=4, marker='square', enable_move=True)
scatter2 = plt.scatter(xs[triangle_inds], ys[triangle_inds], default_size=600, colors=['white'], 
                       stroke='green', stroke_width=4, marker='triangle-up', enable_move=True)

# status_text = plt.label(["Not Yet"], x=[5], y=[11], 
#                             align='middle', font_weight='bold', default_size=24, colors=['Black'])

htmltext = "blah blah"
haha = widgets.HTML(value = f"<b><font color='red'>{htmltext}</b>", 
                          layout=widgets.Layout(border='white solid 1px'))

def randomize_positions(button, scatter0, scatter1, scatter2):
#    print("wer")
    scatter0.x = np.random.uniform(0, xmax, 3)
    scatter1.x = np.random.uniform(0, xmax, 3)
    scatter2.x = np.random.uniform(0, xmax, 3)
    scatter0.y = np.random.uniform(0, ymax, 3)
    scatter1.y = np.random.uniform(0, ymax, 3)
    scatter2.y = np.random.uniform(0, ymax, 3)
    
button_randomize = widgets.Button(
    description='Randomize positions',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description')
    
button_randomize.on_click(lambda button: randomize_positions(button, scatter0, scatter1, scatter2))

def is_simpson(reg0, reg1, reg2, reg012, haha):
    conds = [(reg0.coef_ < 0), (reg1.coef_ < 0), (reg2.coef_ < 0), (reg012.coef_ > 0)]
    if conds[0]:
        line_reg0.set_trait('opacities', [1])
    else:
        line_reg0.set_trait('opacities', [0.2])
        
    if conds[1]:
        line_reg1.set_trait('opacities', [1])
    else:
        line_reg1.set_trait('opacities', [0.2])
        
    if conds[2]:
        line_reg2.set_trait('opacities', [1])
    else:
        line_reg2.set_trait('opacities', [0.2])
        
    if conds[3]:
        line_reg012.set_trait('opacities', [1])
    else:
        line_reg012.set_trait('opacities', [0.2])
      
    if sum(conds[:-1]) == 0:
        #status_text.text = ["Move the shapes"]
        status_text = "Move the shapes"
    elif sum(conds[:-1]) == 1:
        #status_text.text = ["You got one :)"]
        status_text = "You got one :)"
    elif sum(conds[:-1]) == 2:
        #status_text.text = ["You got two!!"]
        status_text = "You got two!!"
    elif sum(conds[:-1]) == 3 and conds[-1] == 0:
        #status_text.text = ["Now for the total!"]
        status_text = "Now for the total!"
    elif sum(conds) == 4:
        #status_text.text = ["You designed a Simpson's Paradox!"]
        status_text = "You designed a Simpson's Paradox!"
    else:
        print('ELSE+++++++++++++++++++++++')

    haha.value = "<b><font color='black'><font size='5em'>" + "{}".format(status_text) + "</b>"
    
def update_reg_line(change, scatter0, scatter1, scatter2, haha):
    reg0 = LinearRegression().fit(scatter0.x.reshape(-1, 1), scatter0.y.reshape(-1, 1))
    reg1 = LinearRegression().fit(scatter1.x.reshape(-1, 1), scatter1.y.reshape(-1, 1))
    reg2 = LinearRegression().fit(scatter2.x.reshape(-1, 1), scatter2.y.reshape(-1, 1))
    
    newxs = np.concatenate([scatter0.x, scatter1.x, scatter2.x])
    newys = np.concatenate([scatter0.y, scatter1.y, scatter2.y])
    reg012 = LinearRegression().fit(newxs.reshape(-1, 1), newys.reshape(-1, 1))
    
    reg_ys0 = reg0.predict(reg_xs.reshape(-1, 1))
    reg_ys1 = reg1.predict(reg_xs.reshape(-1, 1))
    reg_ys2 = reg2.predict(reg_xs.reshape(-1, 1))
    reg_ys012 = reg012.predict(reg_xs.reshape(-1, 1))

    line_reg0.y = reg_ys0
    line_reg1.y = reg_ys1
    line_reg2.y = reg_ys2
    line_reg012.y = reg_ys012
    
    is_simpson(reg0, reg1, reg2, reg012, haha)
    
scatter0.observe(lambda change: update_reg_line(change, scatter0, scatter1, scatter2, haha), names=['x','y'])
scatter1.observe(lambda change: update_reg_line(change, scatter0, scatter1, scatter2, haha), names=['x','y'])
scatter2.observe(lambda change: update_reg_line(change, scatter0, scatter1, scatter2, haha), names=['x','y'])
# scatter1.observe(update_reg_line, names=['x','y'])
# scatter2.observe(update_reg_line, names=['x','y'])

is_simpson(reg0, reg1, reg2, reg012, haha)
    
plt.xlim(0, xmax)
plt.ylim(0, 11)

fig_challenge1.axes[0].set_trait('visible', False)
fig_challenge1.axes[1].set_trait('visible', False)

box_challenge1 = widgets.VBox([haha, fig_challenge1])

box_challenge1.layout.border = 'white 1px solid'
box_challenge1.layout.width = '500px'
box_challenge1.layout.align_items = 'center'

box_challenge1

In [8]:
%%HTML
<style>
td {
  font-size: 24px
}
</style>

In [9]:
# Create batting dataframe

# Set up multi index dataframe for batting stats
years = [2018, 2019]
batting_details = ['hits', 'at-bats', 'BA']

mindex = pd.MultiIndex.from_product([years, batting_details],
                           names=['year', 'stats'])

# This will set dtypes as int. Batting ave will upcast to float later.
bat_df = pd.DataFrame(np.zeros(shape=(2, 6), dtype='int'), index=['Jackie', 'Arlo'], columns=mindex)

########
# Enter the main data: 'hits' and 'at bats'
bat_df.loc['Jackie'] = [1, 20, 0.0, 24, 80, 0.0]
bat_df.loc['Arlo'] = [12, 80, 0.0, 10, 20, 0.0]
########

# TODO: This could be done better.
bat_df.loc[:, ('Total', 'hits')] = bat_df.loc[:, (2018, 'hits')] + bat_df.loc[:, (2019, 'hits')]
bat_df.loc[:, ('Total', 'at-bats')] = bat_df.loc[:, (2018, 'at-bats')] + bat_df.loc[:, (2019, 'at-bats')]

def compute_BA(df):
    for year in df.columns.get_level_values('year').unique():
        df.loc[:, (year,'BA')] = df[year]['hits'] / df[year]['at-bats']
    return df

bat_df = compute_BA(bat_df)

# EXAMPLE 2: Batting Averages (BA)

<hr style="height:6px">

Jackie and Arlo have been playing baseball for the past two years.
Baseball fans argue about who is the better player - in particular, who has the better "batting average" (BA).

In [10]:
# Show stats

seasons_out = widgets.Output()
overall_out = widgets.Output()

with seasons_out:
    display(bat_df[bat_df.columns[0:6]])
    
button_overall_stats = widgets.ToggleButton(description='Check overall stats', value=False)

def on_button_overall_stats(change):
    with overall_out:
        if change['new'] == True:
            display(bat_df[bat_df.columns[6:]])
        else:
            overall_out.clear_output()
        
button_overall_stats.observe(on_button_overall_stats, 'value')

#stats_box = widgets.HBox([seasons_out, button_overall_stats, overall_out])
stats_box = widgets.HBox([seasons_out, overall_out])
stats_box.layout.align_items = 'center'

stats_box

The batting average is defined as the ratio of the number of **hits** to the number of **at-bats**.

For example, the table shows that in 2018 Jackie had only 1 hit in 20 at-bats.
Therefore his batting average for 2018 is:

$$\textrm{BA} = \frac{\textrm{hits}}{\textrm{at-bats}} \quad \quad \quad BA_\textrm{Jackie 2018} = \frac{1}{20} = 0.05$$

In [11]:
md_str = """Some fans say "Arlo is definitely the better player! He had a better batting average in each of the two seasons.
(Look at the table and confirm that this is true.)

Other fans say, "Yeah, but what about the *overall batting average*?"
(Take a minute and compute each player's overall BA. This is the total number of hits divided by the total number of at-bats.)
"""

display(Markdown(md_str))

button_overall_stats

Some fans say "Arlo is definitely the better player! He had a better batting average in each of the two seasons.
(Look at the table and confirm that this is true.)

Other fans say, "Yeah, but what about the *overall batting average*?"
(Take a minute and compute each player's overall BA. This is the total number of hits divided by the total number of at-bats.)


ToggleButton(value=False, description='Check overall stats')

Hopefully you found that while Arlo has a better BA in each season, Jackie has the better total BA. 

<p style="font-size:1.5em"> How can this be? </p>

We have another great example of Simpson's Paradox!
Let's try to gain some intuition by graphing the data.

In [12]:
# slice(None) important for slicing a multi index
hits = bat_df.loc[:, (slice(None), 'hits')].to_numpy()
atbats = bat_df.loc[:, (slice(None), 'at-bats')].to_numpy()

# We want to plot the cumulative.
# -1 because we don't want the Total
hitscum = np.cumsum(hits[:, :-1], axis=1)
atbatscum = np.cumsum(atbats[:, :-1], axis=1)

# Prepend 0s for plotting
hitscum = np.hstack((np.zeros(shape=(2,1)), hitscum))
atbatscum = np.hstack((np.zeros(shape=(2,1)), atbatscum))

#print(hitscum)
#print(atbatscum)

In [13]:
# Make batting average interactive!

# Color choices
jackie_colors = ['red', 'red']
arlo_colors = ['blue', 'blue']

fig_batting = plt.figure(animation_duration=1000, 
                         min_aspect_ratio=1.3, max_aspect_ratio=1.31, 
                         fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})
fig_batting.layout.width = '100%'
fig_batting.layout.height = '400px'
fig_batting.layout.border = 'black solid 1px'

jackie_hits_cum = hitscum[0, :]
jackie_atbats_cum = atbatscum[0, :]

arlo_hits_cum = hitscum[1, :]
arlo_atbats_cum = atbatscum[1, :]

jackie_line_overall = plt.plot(x=jackie_atbats_cum[[0, -1]], y=jackie_hits_cum[[0, -1]], colors=['lightblue'], 
                               stroke_width=2, visible=False)
arlo_line_overall = plt.plot(x=arlo_atbats_cum[[0, -1]], y=arlo_hits_cum[[0, -1]], colors=['pink'], 
                             stroke_width=2, visible=False)

jackie_lines = []
arlo_lines = []
for ind in range(2):
    jackie_lines.append(plt.plot(x=atbatscum[0, [ind, ind+1]], y=hitscum[0, [ind, ind+1]], colors=[jackie_colors[ind]], 
                       stroke_width=5, line_style='solid'))
    arlo_lines.append(plt.plot(x=atbatscum[1, [ind, ind+1]], y=hitscum[1, [ind, ind+1]], colors=[arlo_colors[ind]], 
                       stroke_width=5, line_style='solid'))

jackie_lines[1].set_trait('line_style', 'dashed')
arlo_lines[1].set_trait('line_style', 'dashed')
jackie_lines[0].set_trait('labels', [''])
arlo_lines[0].set_trait('labels', [''])
jackie_lines[1].set_trait('labels', ['Jackie'])
arlo_lines[1].set_trait('labels', ['Arlo'])

vert_lines = []
for ind in [0, 1]:
    vert_lines.append(plt.plot(x=2*[atbatscum[ind, 1]], y=[0, 25], colors=['black'], line_style='solid', 
                       stroke_width=1))

jackie_text = plt.label(["Jackie"], x=[35], y=[20], align='middle', font_weight='bold', default_size=24, 
                      colors=['Red'])
arlo_text = plt.label(["Arlo"], x=[65], y=[20], align='middle', font_weight='bold', default_size=24, 
                      colors=['Blue'])


plt.xlabel("at-bats")
plt.ylabel("hits")

button_bat1 = widgets.ToggleButtons(
    value='season',
    options=['season', 'at-bats'],
    description='Sort by:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

button_overall = widgets.ToggleButton(
    value=False,
    description='Show overall',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description')
   
def switch_vals(arrA, arrB):
    temp = copy.copy(arrA)
    arrA = arrB
    arrB = temp
    return arrA, arrB

def on_button_bat1(change):
    # TODO: This function is rather terrible, but abstracting seems like a big pain right now.
    # Switch Arlo lines
    dx0 = np.diff(arlo_lines[0].x)[0]
    dy0 = np.diff(arlo_lines[0].y)[0]
    
    dx1 = np.diff(arlo_lines[1].x)[0]
    dy1 = np.diff(arlo_lines[1].y)[0]
    
    if change['new'] == 'at-bats':
        arlo_lines[1].x = [0, dx1]
        arlo_lines[1].y = [0, dy1]

        arlo_lines[0].x = [dx1, dx0 + dx1]
        arlo_lines[0].y = [dy1, dy0 + dy1]
        
    elif change['new'] == 'season':
        arlo_lines[0].x = [0, dx0]
        arlo_lines[0].y = [0, dy0]

        arlo_lines[1].x = [dx0, dx0 + dx1]
        arlo_lines[1].y = [dy0, dy0 + dy1]
    else:
        print(change)
    
    # Move second vert line
    if change['new'] == 'at-bats':
        vert_lines[1].x = x=2*[atbatscum[0, 1]]
    elif change['new'] == 'season':
        vert_lines[1].x = x=2*[atbatscum[1, 1]]
    else:
        print(change)
    
def on_button_overall(change):
    if change['new'] == True:
        jackie_line_overall.visible = True
        arlo_line_overall.visible = True
    else:
        jackie_line_overall.visible = False
        arlo_line_overall.visible = False

button_bat1.observe(on_button_bat1, names='value')

button_overall.observe(on_button_overall, names='value')

button_box = widgets.VBox([button_bat1, button_overall])
button_box.layout.justify_content = 'space-around'
button_box.layout.align_items = 'center'
button_box.layout.height = '200px'
button_box.layout.border = 'white solid 1px'

box_bat = widgets.HBox([fig_batting, button_box])
box_bat.layout.border = 'black solid 1px'
box_bat.layout.width = '800px'
box_bat.layout.height = '100%'
box_bat.layout.align_items = 'center'

box_bat

Here, we show hits vs at bats.
The solid lines represent the first season and the dashed lines represent the second season.
For example, the solid red line tells us that Jackie had only 1 hit in 20 at-bats in 2018.

The slope of each line segment represents the batting average for that period.
> We can see from the graph then that Arlo has a better BA for each season.

We can also graph at the overall statistics.
(Try the "Show Overall" button.)
These light-colored lines represent the overall hits and at-bats for each player.
And so the slope of this line represents their overall BA.
> We can see that Jackie has a better overall BA.

<div class="alert alert-info" role="alert">
    <p style="font-size:1.5em"> Leading the batting average in each season is not enough to ensure you will lead overall.</p>
</div>

How can Arlo be better for each season, but Jackie be better overall?

We are confronted with this numerical fact, but it seems wrong somehow.

Let's try reorganizing the data: instead of sorting by season, let's sort by comparable at-bats.
From this vantage, Arlo still leads in one group (20 at-bats) but Jackie leads in the other (80 at-bats).
Since the leadership is now mixed, it should be no surprise that either player might attain the higher overall BA.

<div class="alert alert-info" role="alert">
    <p style="font-size:1.5em"> When we compare "apples to apples" (comparable number of at-bats), the paradox is explained.</p>
</div>

## QUICK QUIZ

<div class="alert alert-success" role="alert">
</div>

Let's explore an alternate version of this scenario.
What if Jackie and Arlo *each* had 20 at-bats in 2018 and 80 at-bats in 2019.
Is Simpson’s Paradox possible?

Let's switch gears and do a little algebra:

Imagine Jackie has $a$ hits in 2018 and $b$ hits 2019.
Similarly, Arlo has $c$ and $d$ hits respectively.

Assume that Arlo leads in batting average for each season:

Which of the following are true? (Check all that apply)

- $a / 20 < c / 20$ (+)
- $b / 80 < d / 80$ (+)
- $a < c$ (+)
- $b < d$ (+)

Jackie's overall average is $(a + b) / 100$ and Arlo's overall average is $(c + d) / 100$.

Putting this all together, which of the following are true? (Check all that apply)

- a + b  < c + d (+)
- a + c  < b + d 

Finally, $(a + b) / 100 < (c + d) / 100$ - Arlo must the overall leader, and there can be no Simpson’s Paradox!

<div class="alert alert-info" role="alert">
    <p style="font-size:1.5em"> Simpson's Paradox depends on an imbalance in the number of at-bats between players.</p>
</div>

# Interactive Challenge!

Now consider two new baseball players, Lian and Jose.
Imagine they have played for three seasons.
Can you design a scenario where Lian has a better BA for each season, but Jose has the better overall BA?
As you move the dots, notice the changes in the leaderboard.
(Hint: Remember that we need an imbalance in at-bats...)

In [14]:
# TODO add axis labels
# Make interactive batting challenge

# TODO: Make a toggle or second challenge or just quiz question where the at-bats for each section is the same for both players.

# fig_interactive_batting = plt.figure(layout=widgets.Layout(width='auto', height='auto'), animation_duration=100, 
#                                      fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20}, display_legend=True)

fig_interactive_batting = plt.figure(animation_duration=100, 
                         min_aspect_ratio=1.3, max_aspect_ratio=1.31, 
                         fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

fig_interactive_batting.layout.width = '100%'
fig_interactive_batting.layout.height = '400px'
fig_interactive_batting.layout.border = 'black solid 1px'

lian_hits_cum = np.array([0, 10, 20, 30])
lian_atbats_cum = np.array([0, 20, 40, 60])

jose_hits_cum = np.array([0, 12, 24, 36])
jose_atbats_cum = np.array([0, 20, 40, 60])

lian_lineA = plt.plot(x=lian_atbats_cum[0:2], y=lian_hits_cum[0:2], colors=['blue'], stroke_width=5)
lian_lineB = plt.plot(x=lian_atbats_cum[1:3], y=lian_hits_cum[1:3], colors=['blue'], stroke_width=5, line_style='dashed')
lian_lineC = plt.plot(x=lian_atbats_cum[2:4], y=lian_hits_cum[2:4], colors=['blue'], stroke_width=5, line_style='dotted')
jose_lineA = plt.plot(x=jose_atbats_cum[0:2], y=jose_hits_cum[0:2], colors=['red'], stroke_width=5)
jose_lineB = plt.plot(x=jose_atbats_cum[1:3], y=jose_hits_cum[1:3], colors=['red'], stroke_width=5, line_style='dashed')
jose_lineC = plt.plot(x=jose_atbats_cum[2:4], y=jose_hits_cum[2:4], colors=['red'], stroke_width=5, line_style='dotted')

lian_line_overall = plt.plot(x=lian_atbats_cum[[0, -1]], y=lian_hits_cum[[0, -1]], colors=['lightblue'], stroke_width=3)
jose_line_overall = plt.plot(x=jose_atbats_cum[[0, -1]], y=jose_hits_cum[[0, -1]], colors=['pink'], stroke_width=3)

lian_scatter0 = plt.scatter(x=[lian_atbats_cum[0]], y=[lian_hits_cum[0]], colors=['blue'], default_size=200, enable_move=False)
jose_scatter0 = plt.scatter(x=[jose_atbats_cum[0]], y=[jose_hits_cum[0]], colors=['red'], default_size=200, enable_move=False)

lian_scatter = plt.scatter(x=lian_atbats_cum[1:], y=lian_hits_cum[1:], colors=['blue'], default_size=200, enable_move=True)
jose_scatter = plt.scatter(x=jose_atbats_cum[1:], y=jose_hits_cum[1:], colors=['red'], default_size=200, enable_move=True)


lian_text = plt.label(["Lian"], x=[10], y=[30], align='middle', font_weight='bold', default_size=24, 
                      colors=['Blue'])
jose_text = plt.label(["Jose"], x=[25], y=[30], align='middle', font_weight='bold', default_size=24, 
                      colors=['Red'])


plt.xlabel("at-bats")
plt.ylabel("hits")

htmltext = "blah blah"
htmlWidget = widgets.HTML(value = f"<b><font color='red'>{htmltext}</b>")

def on_lian_move(change):
    if change['name'] == 'x':
        newxs = change['new']
        allnewxs = np.insert(newxs, 0, lian_scatter0.x[0])
        
        if np.any(np.diff(allnewxs) < 0):
            # Reject the move
            lian_scatter.x = change['old']
            return
        
        lian_lineA.x = allnewxs[0:2]
        lian_lineB.x = allnewxs[1:3]
        lian_lineC.x = allnewxs[2:4]
        lian_line_overall.x = allnewxs[[0, -1]]
        
    if change['name'] == 'y':
        newys = change['new']
        allnewys = np.insert(newys, 0, lian_scatter0.y[0])
        
        if np.any(np.diff(allnewys) < 0):
            # Reject the move
            lian_scatter.y = change['old']
            return
        
        lian_lineA.y = allnewys[0:2]
        lian_lineB.y = allnewys[1:3]
        lian_lineC.y = allnewys[2:4]
        lian_line_overall.y = allnewys[[0, -1]]

    is_simpson_bat(lian_lineA, lian_lineB, lian_lineC, jose_lineA, jose_lineB, jose_lineC)
    
def on_jose_move(change):
    if change['name'] == 'x':
        newxs = change['new']
        allnewxs = np.insert(newxs, 0, jose_scatter0.x[0])

        if np.any(np.diff(allnewxs) < 0):
            # Reject the move
            jose_scatter.x = change['old']
            return
        
        jose_lineA.x = allnewxs[0:2]
        jose_lineB.x = allnewxs[1:3]
        jose_lineC.x = allnewxs[2:4]
        jose_line_overall.x = allnewxs[[0, -1]]
        
    if change['name'] == 'y':
        newys = change['new']
        allnewys = np.insert(newys, 0, jose_scatter0.y[0])
        
        if np.any(np.diff(allnewys) < 0):
            # Reject the move
            jose_scatter.y = change['old']
            return
        
        jose_lineA.y = allnewys[0:2]
        jose_lineB.y = allnewys[1:3]
        jose_lineC.y = allnewys[2:4]
        jose_line_overall.y = allnewys[[0, -1]]
    
    is_simpson_bat(lian_lineA, lian_lineB, lian_lineC, jose_lineA, jose_lineB, jose_lineC)

def is_slope_greater(lineA, lineB):
    slopeA = np.diff(lineA.y) / np.diff(lineA.x)
    slopeB = np.diff(lineB.y) / np.diff(lineB.x)
    if slopeA > slopeB:
        return True
    else:
        return False
    
def is_simpson_bat(lian_lineA, lian_lineB, lian_lineC, jose_lineA, jose_lineB, jose_lineC):
    
    conds = [is_slope_greater(lian_lineA, jose_lineA), 
             is_slope_greater(lian_lineB, jose_lineB), 
             is_slope_greater(lian_lineC, jose_lineC), 
             is_slope_greater(lian_line_overall, jose_line_overall)]
    
    names = ["<font color='blue'>Lian", "<font color='red'>Jose"]
    if conds[0]:
        nameA = names[0]
    else:
        nameA = names[1]
        
    if conds[1]:
        nameB = names[0]
    else:
        nameB = names[1]
        
    if conds[2]:
        nameC = names[0]
    else:
        nameC = names[1]
        
    if conds[3]:
        name_overall = names[0]
    else:
        name_overall = names[1]
        
    if conds[0] and conds[1] and conds[2] and not conds[3]:
        congratulations = 'Congratulations!'
    else:
        congratulations = ''

    htmlWidget.value =  "" + \
    """
    <table style='width:200px;font-size:2.0em'>
      <tr>
        <th>Year</th>
        <th>Leader</th>
      </tr>
      <br>
      <tr>
        <td>2020</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>2021</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>2022</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>Overall</td>
        <td>{}</td>
      </tr>
    </table>
    <br>
    <p style="font-size:2em">{}</p>
    """.format(nameA, nameB, nameC, name_overall, congratulations)
        
lian_scatter.observe(on_lian_move, names=['x', 'y'])
jose_scatter.observe(on_jose_move, names=['x', 'y'])

is_simpson_bat(lian_lineA, lian_lineB, lian_lineC, jose_lineA, jose_lineB, jose_lineC)

box_interactive_batting = widgets.HBox([fig_interactive_batting, htmlWidget])
box_interactive_batting.layout.border = 'black solid 1px'
box_interactive_batting.layout.width = '800px'

box_interactive_batting

# EXAMPLE 3: Red Triangles

<hr style="height:6px">

Below we see ten objects divided into two groups of five.
Each object has a color (red or blue) and a shape (cross or triangle), but these details are hidden from us.
Initially, we are presented with gray squares.

**Your goal is to find the group (upper/lower) with the most red triangles.**

The catch is that we are allowed to reveal either the shape or color, but not both at once.

- First, use the reveal button to see the shapes - which group has more triangles?
- Then use the button to reveal colors - which group has more red objects?
- Now make your choice! Which group do you suspect has the most red triangles?
- Reveal the "shape AND color" to see if you guessed right.

In [15]:
def red_triangles_interactive():
    """
    This interactive displays a flavor of Simpson's Paradox.
    
    The user is asked whether there are more red triangles on the top or bottom level.
    
    In this interactive, the user can reveal either the color or the shape but not both.
    Based on this information they must make a choice of level.
    After they have chosen, they may reveal both the color and shape.
    If they choose "rationally", they will find that their choice is wrong.
    
    It is based on the the example found in the Martin Gardner book.
    Sometimes this example is seen as being about a woman looking for a man that is kind AND rich.
    She examines the groups bald men and notbald men.
    The notbald men are more likely to be kind.
    The notbald men are also more likely to be rich.
    However, because of the (anti) correlation between these features, the bald men are more likely to be kind AND rich.
    """
    
    # Each object has a position x, y and a color.
    num = 10
    colors = ['red', 'blue']
    shapes = ['triangle-up', 'cross']

    # This `df_init` remains untouched
    df_init = pd.DataFrame({'x':np.arange(num) % 5, 'y': [0 for ind in range(num//2)] + [1 for ind in range(num//2)], 
                       'color':[0, 0, 1, 0, 1, 1, 1, 0, 0, 1], 
                       'shape':[0, 1, 0, 1, 0, 1, 1, 0, 0, 1]})

    df_init['color'] = df_init['color'].map(dict(zip(np.arange(num), colors)))
    df_init['shape'] = df_init['shape'].map(dict(zip(np.arange(num), shapes)))

    # This dataframe `df` will be modified as we go.
    df = copy.copy(df_init)

    def get_shape_dfs(df):
        dfA = df[df['shape']==shapes[0]]
        dfB = df[df['shape']==shapes[1]]
        return dfA, dfB

    def permute_objects_in_df(df):
        n = len(df)
        halfn = n//2
        inds0 = np.random.choice(np.arange(halfn), replace=False, size=halfn)
        inds1 = np.random.choice(np.arange(halfn), replace=False, size=halfn) + halfn
        allinds = np.append(inds0, inds1)

        permuted_xs = df['x'][allinds].to_numpy()
        permuted_ys = df['y'][allinds].to_numpy() # Note that because we are permuting within rows this does nothing.

        df['x'] = permuted_xs
        df['y'] = permuted_ys

    def sync_scatters_position_w_df(scatterA, scatterB, df):
        # This can sync the scatter plots with either the initial or the active df.
    #     print("in sync")
        dfA, dfB = get_shape_dfs(df)

        scatterA.set_trait('x', dfA['x'].to_numpy())
        scatterA.set_trait('y', dfA['y'].to_numpy())

        scatterB.set_trait('x', dfB['x'].to_numpy())
        scatterB.set_trait('y', dfB['y'].to_numpy())
        
    def sync_scatters_w_df(scatterA, scatterB, df):
        # This is intended as a hard reset

        dfA, dfB = get_shape_dfs(df)

        scatterA.set_trait('x', dfA['x'].to_numpy())
        scatterA.set_trait('y', dfA['y'].to_numpy())

        scatterB.set_trait('x', dfB['x'].to_numpy())
        scatterB.set_trait('y', dfB['y'].to_numpy())
        
        show_color(scatterA, scatterB, df)
        show_shape(scatterA, scatterB)
        
    def on_button_reset(scatterA, scatterB, df, button_reveal, button_choice):
        print("reset")
        sync_scatters_w_df(scatterA, scatterB, df)
        
        # FIX: This apparently does not remove the "color and shape" option and I don't know why not.
        button_reveal.options = ['nothing', 'color', 'shape']
        button_reveal.value = 'nothing'
        
        button_choice.value = None

    def permute_scatter_positions(scatterA, scatterB, df):
        # Permute but within each row
        # We have to operate on both shape types at once.
    #     print("permute scatter position")

        permute_objects_in_df(df)
        sync_scatters_position_w_df(scatterA, scatterB, df)

    def hide_color(scatterA, scatterB):
        scatterA.colors = ['gray']
        scatterB.colors = ['gray']

    def show_color(scatterA, scatterB, df):
        dfA, dfB = get_shape_dfs(df)
        scatterA.colors = list(dfA['color'])
        scatterB.colors = list(dfB['color'])

    def hide_shape(scatterA, scatterB):
        scatterA.set_trait('marker', 'square')
        scatterB.set_trait('marker', 'square')

    def show_shape(scatterA, scatterB):
        scatterA.set_trait('marker', 'triangle-up')
        scatterB.set_trait('marker', 'cross')

    def on_button_choice(change, button_reveal):
    #     print("button choice")
        curr_val = button_reveal.value
        button_reveal.set_trait('options', ('nothing', 'color', 'shape', 'color and shape'))
        button_reveal.value = curr_val

    def on_button_reveal(change, scatterA, scatterB, df):
        # Reveal the requested properties.
        # Shape AND color only accessible after a level has been chosen

        state = change['new']

        if state == 'nothing':
            # Hide color and shape.
            hide_color(scatterA, scatterB)
            hide_shape(scatterA, scatterB)
        elif state == 'color':
            # Hide shape and show color.
            show_color(scatterA, scatterB, df)
            hide_shape(scatterA, scatterB)
        elif state == 'shape':
            # Hide color and show shape.
            hide_color(scatterA, scatterB)
            show_shape(scatterA, scatterB)
        elif state == 'color and shape':
            show_color(scatterA, scatterB, df)
            show_shape(scatterA, scatterB)
        else:
            print("not implemented yet")

        # If we have made a decision, don't continue to permute.
        # We want the user to see how their guess corresponds to reality without any extra confusion.
        if button_choice.value is None:
            permute_scatter_positions(scatterA, scatterB, df)

    # Make a figure showing these objects

    fig = plt.figure(animation_duration=1000, 
                     min_aspect_ratio=2.5, max_aspect_ratio=2.51, 
                     fig_margin={'top':20, 'bottom':20, 'left':20, 'right':20})
    fig.layout.width = '400px'
    fig.layout.height = '200px'
    fig.layout.border = 'black solid 1px'
    fig.layout.align_items = 'center'
    
#     fig = plt.figure(ax=[], layout={'height':'300px', 'width':'500px', 'border':'black solid 2px'}, animation_duration=1000)

    dfA, dfB = get_shape_dfs(df)

    scatterA = plt.scatter(dfA['x'], dfA['y'], colors=list(dfA['color']), marker=shapes[0], default_size=1000)
    scatterB = plt.scatter(dfB['x'], dfB['y'], colors=list(dfB['color']), marker=shapes[1], default_size=1000)

    div_line = plt.plot([-0.5, 4.5], [0.5, 0.5], colors=['black'])

    # Create the buttons
    button_choice = widgets.RadioButtons(description="Choose:", options=['top', 'bottom'], value=None, 
                                         layout={'width':'160px'})

    button_reveal = widgets.Dropdown(description="Reveal:", options=['nothing', 'color', 'shape'], value='nothing', 
                                    layout={'border':'white solid 1px', 'width':'225px'})

    button_permute_positions = widgets.Button(description='permute positions')
    button_reset = widgets.Button(description='reset')

    # I think on_click wants a `change` - use lambda to make a dummy variable.
    button_choice.observe(lambda change: on_button_choice(change, button_reveal), 'value')
    button_reveal.observe(lambda change: on_button_reveal(change, scatterA, scatterB, df), 'value')

    button_permute_positions.on_click(lambda change: permute_scatter_positions(scatterA, scatterB, df))
    button_reset.on_click(lambda change: on_button_reset(scatterA, scatterB, df_init, button_reveal, button_choice))

    fig.axes[0].visible = False
    fig.axes[1].visible = False

    hide_color(scatterA, scatterB)
    hide_shape(scatterA, scatterB)

    rightbox = widgets.VBox([button_reveal, button_reset],
                             layout={'border':'white solid 1px', 'height':'100px'})
    rightbox.layout.align_items = 'center'
    rightbox.layout.justify_content = 'space-around'
    
#                                 pane_widths=[2, 3, 1.5], 
    box = widgets.AppLayout(left_sidebar=button_choice, center=fig, right_sidebar=rightbox, 
                            layout=widgets.Layout(border='black solid 1px', 
                                                  width='auto',
                                                  align_items='center'),
                            grid_gap='0px')
    box.layout.width = '800px'
    
    return box

In [16]:
red_triangles_box = red_triangles_interactive()
red_triangles_box

AppLayout(children=(RadioButtons(description='Choose:', layout=Layout(grid_area='left-sidebar', width='160px')…

## QUICK QUIZ:

- Which group has more triangles? (bottom)

- Which group has more red shapes? (bottom)

- Which group has the most red triangles? (top)

- On the top, "red" and "triangle" are _____ correlated. (positively)

- On the bottom, "red" and "triangle" are _____ correlated. (negatively)

<div class="alert alert-info" role="alert">
    <p style="font-size:1.5em"> The two levels have opposing correlations between "red" and "triangle".</p>
</div>

# Application Challenge!

<hr style="height:6px">

Let's apply our new understanding of Simpson's Paradox to an important situation we are all likely to encounter:
When presented with complicated health data, how do you make the best decision?

Imagine there are two new vaccines (vaccine A, vaccine B) that have just undergone early clinical trials.

Your friend reads something curious in the news:
"This article reports that vaccine A had a higher rate of success than vaccine B in both the trials on males and the trials on females. But another article reports that vaccine B had a higher rate of success than vaccine A overall."

"This can't be true? Can it?!", they say.

Use your new intuition and the interactive tool below to show your friend how this counter-intuitive situation is indeed possible.

In [17]:
# Drug interactive
# TODO: set initial data
# TODO: prob fonts larger

# I could do this with pandas using a Multi index, but I'm going to just get 'er done with numpy.

# Create the data grid
grid = widgets.GridspecLayout(7, 7)

grid[2, 0] = widgets.Label(value=r"\(\color{red}{Vaccine A}\)")
grid[3, 0] = widgets.Label(value=r"\(\color{blue}{Vaccine B}\)")

grid[0, 1:3] = widgets.Label(value="--- Males ---")
grid[0, 1:3].layout.display = 'flex'
grid[0, 1:3].layout.justify_content = 'center'
grid[0, 1:3].layout.border = 'black solid 1px'

grid[0, 3:5] = widgets.Label(value="--- Females ---")
grid[0, 3:5].layout.display = 'flex'
grid[0, 3:5].layout.justify_content = 'center'
grid[0, 3:5].layout.border = 'black solid 1px'

grid[0, 5:7] = widgets.Label(value="--- All People ---")
grid[0, 5:7].layout.display = 'flex'
grid[0, 5:7].layout.justify_content = 'center'
grid[0, 5:7].layout.border = 'black solid 1px'

grid[1, 1] = widgets.Label(value="Cured")
grid[1, 2] = widgets.Label(value="Not cured")
grid[1, 3] = widgets.Label(value="Cured")
grid[1, 4] = widgets.Label(value="Not cured")
grid[1, 5] = widgets.Label(value="Cured")
grid[1, 6] = widgets.Label(value="Not cured")

# Create adjustable cells - male and female
for i in [2,3]:
    for j in [1,2,3,4]:
        grid[i, j] = widgets.IntText(value=0, description='')
        grid[i, j].layout.width = '100px'

# Create read-only cells - total
for i in [2,3]:
    for j in [5, 6]:
        grid[i, j] = widgets.Label(value="0", description='')
        grid[i, j].layout.width = '100px'
        
# Set the data to something desired
vaccine_A_init = [2, 3, 2, 0]
vaccine_B_init = [0, 2, 3, 2]

grid[5, 1:3] = widgets.HTML(value='')
grid[5, 1:3].layout.width = '60%'
grid[6, 1:3] = widgets.HTML(value='')
grid[5, 1:3].layout.width = '60%'

grid[5, 3:5] = widgets.HTML(value='')
grid[5, 1:3].layout.width = '60%'
grid[6, 3:5] = widgets.HTML(value='')
grid[5, 1:3].layout.width = '60%'

grid[5, 5:7] = widgets.HTML(value='')
grid[5, 1:3].layout.width = '60%'
grid[6, 5:7] = widgets.HTML(value='')
grid[5, 1:3].layout.width = '60%'

for j in [1,2,3,4]:
    grid[2, j].value = str(vaccine_A_init[j-1])
    grid[3, j].value = str(vaccine_B_init[j-1])

        
def ratio_string(cellA, cellB):
    return r"{} / {}".format(cellA.value, (int(cellA.value) + int(cellB.value)))

def ratio(cellA, cellB):
    return cellA.value / (cellA.value + cellB.value)

def update_probabilities(grid):
    
    male_bool, female_bool, total_bool = check_inequalities(grid)
    
    color_A_on = 'red'
    color_B_on = 'blue'
    color_A_off = 'black'
    color_B_off = 'black'
    
    if male_bool is True:
        cA = color_A_on
        cB = color_B_off
    elif male_bool is False:
        cA = color_A_off
        cB = color_B_on
    elif male_bool is None:
        cA = color_A_on
        cB = color_B_on
    else:
        raise
        
    # Display probs
    s = 'Pr(Cure | A) = ' + ratio_string(grid[2, 1], grid[2, 2])
    grid[5, 1:3].value = f"<div style='width:100%;height:100%;color:{cA}'> {s} </div>"
    
    s = 'Pr(Cure | B) = ' + ratio_string(grid[3, 1], grid[3, 2])
    grid[6, 1:3].value = f"<div style='width:100%;height:100%;color:{cB}'> {s} </div>"
    
    if female_bool is True:
        cA = color_A_on
        cB = color_B_off
    elif female_bool is False:
        cA = color_A_off
        cB = color_B_on
    elif female_bool is None:
        cA = color_A_on
        cB = color_B_on
    else:
        raise
        
    # Display probs
    s = 'Pr(Cure | A) = ' + ratio_string(grid[2, 3], grid[2, 4])
    grid[5, 3:5].value = f"<div style='width:100%;height:100%;color:{cA}'> {s} </div>"
    
    s = 'Pr(Cure | B) = ' + ratio_string(grid[3, 3], grid[3, 4])
    grid[6, 3:5].value = f"<div style='width:100%;height:100%;color:{cB}'> {s} </div>"

    if total_bool is True:
        cA = color_A_on
        cB = color_B_off
    elif total_bool is False:
        cA = color_A_off
        cB = color_B_on
    elif total_bool is None:
        cA = color_A_on
        cB = color_B_on
    else:
        raise
    
    s = 'Pr(Cure | A) = ' + ratio_string(grid[2, 5], grid[2, 6])
    grid[5, 5:7].value = f"<div style='width:100%;height:100%;color:{cA}'> {s} </div>"
    
    s = 'Pr(Cure | B) = ' + ratio_string(grid[3, 5], grid[3, 6])
    grid[6, 5:7].value = f"<div style='width:100%;height:100%;color:{cB}'> {s} </div>"
    
def update_totals(grid):
    grid[2, 5].value = str(grid[2, 1].value + grid[2, 3].value)
    grid[2, 6].value = str(grid[2, 2].value + grid[2, 4].value)
    grid[3, 5].value = str(grid[3, 1].value + grid[3, 3].value)
    grid[3, 6].value = str(grid[3, 2].value + grid[3, 4].value)   
    
def get_plot_data(grid):
    # Cured and not cured
    vaccine_A_line_data_x = np.array([grid[2, ind].value for ind in [2, 4]])
    vaccine_A_line_data_y = np.array([grid[2, ind].value for ind in [1, 3]])
    vaccine_B_line_data_x = np.array([grid[3, ind].value for ind in [2, 4]])
    vaccine_B_line_data_y = np.array([grid[3, ind].value for ind in [1, 3]])

    # Add cured and not cured to get treated
    vaccine_A_line_data_x += vaccine_A_line_data_y
    vaccine_B_line_data_x += vaccine_B_line_data_y
    
    # Show cumulative to mirror the batting average plot
    vaccine_A_line_data_x = np.cumsum(vaccine_A_line_data_x)
    vaccine_A_line_data_y = np.cumsum(vaccine_A_line_data_y)
    vaccine_B_line_data_x = np.cumsum(vaccine_B_line_data_x)
    vaccine_B_line_data_y = np.cumsum(vaccine_B_line_data_y)
    
    # Start at zero
    vaccine_A_line_data_x = np.append([0], vaccine_A_line_data_x)
    vaccine_A_line_data_y = np.append([0], vaccine_A_line_data_y)
    vaccine_B_line_data_x = np.append([0], vaccine_B_line_data_x)
    vaccine_B_line_data_y = np.append([0], vaccine_B_line_data_y)
    
    return vaccine_A_line_data_x, vaccine_A_line_data_y, vaccine_B_line_data_x, vaccine_B_line_data_y
    
def update_plot(change, grid, vaccine_A_line, vaccine_B_line, vaccine_A_line_total, vaccine_B_line_total):
    
    vaccine_A_line_data_x, vaccine_A_line_data_y, vaccine_B_line_data_x, vaccine_B_line_data_y = get_plot_data(grid)
    
    vaccine_A_line.x = vaccine_A_line_data_x
    vaccine_A_line.y = vaccine_A_line_data_y

    vaccine_A_scatter.x = vaccine_A_line_data_x
    vaccine_A_scatter.y = vaccine_A_line_data_y

    vaccine_B_line.x = vaccine_B_line_data_x
    vaccine_B_line.y = vaccine_B_line_data_y
    
    vaccine_B_scatter.x = vaccine_B_line_data_x
    vaccine_B_scatter.y = vaccine_B_line_data_y
    
    vaccine_A_line_total.x = vaccine_A_line_data_x[[0, -1]]
    vaccine_A_line_total.y = vaccine_A_line_data_y[[0, -1]]
    
    vaccine_B_line_total.x = vaccine_B_line_data_x[[0, -1]]
    vaccine_B_line_total.y = vaccine_B_line_data_y[[0, -1]]

def check_inequalities(grid):
    # TODO: add the overall test
    # males
    prob_A_cure_males = grid[2, 1].value / (grid[2, 1].value + grid[2, 2].value)
    prob_B_cure_males = grid[3, 1].value / (grid[3, 1].value + grid[3, 2].value)
    
    prob_A_cure_females = grid[2, 3].value / (grid[2, 3].value + grid[2, 4].value)
    prob_B_cure_females = grid[3, 3].value / (grid[3, 3].value + grid[3, 4].value)
    
    prob_A_cure_total = (grid[2, 1].value + grid[2, 3].value) / \
                        (grid[2, 1].value + grid[2, 2].value + grid[2, 3].value + grid[2, 4].value)
    prob_B_cure_total = (grid[3, 1].value + grid[3, 3].value) / \
                        (grid[3, 1].value + grid[3, 2].value + grid[3, 3].value + grid[3, 4].value)  
    
    if prob_A_cure_males > prob_B_cure_males:
        male_bool = True
    elif prob_A_cure_males == prob_B_cure_males:
        male_bool = None
    else:
        male_bool = False
        
    if prob_A_cure_females > prob_B_cure_females:
        female_bool = True
    elif prob_A_cure_females == prob_B_cure_females:
        female_bool = None
    else:
        female_bool = False
        
    if prob_A_cure_total > prob_B_cure_total:
        total_bool = True
    elif prob_A_cure_total == prob_B_cure_total:
        total_bool = None
    else:
        total_bool = False
        
    return male_bool, female_bool, total_bool
    
def on_text_update(change, grid):
    # Don't worry which is which, just recompute everything
    update_probabilities(grid)
    update_totals(grid)
    

fig_vaccines = plt.figure(animation_duration=1000, min_aspect_ratio=2.0, max_aspect_ratio=2.01, 
                       fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

fig_vaccines.layout.width = '600px'
fig_vaccines.layout.height = '400px'
fig_vaccines.layout.border = 'black solid 1px'

vaccine_A_line_data_x, vaccine_A_line_data_y, vaccine_B_line_data_x, vaccine_B_line_data_y = get_plot_data(grid)

vaccine_A_line_total = plt.plot(vaccine_A_line_data_x[[0, -1]], vaccine_A_line_data_y[[0, -1]], colors=['pink'], stroke_width=4)
vaccine_B_line_total = plt.plot(vaccine_B_line_data_x[[0, -1]], vaccine_B_line_data_y[[0, -1]], colors=['lightblue'], stroke_width=4)

vaccine_A_line = plt.plot(vaccine_A_line_data_x, vaccine_A_line_data_y, colors=['red'], stroke_width=4)
vaccine_B_line = plt.plot(vaccine_B_line_data_x, vaccine_B_line_data_y, colors=['blue'], stroke_width=4)

vaccine_A_scatter = plt.scatter(vaccine_A_line_data_x, vaccine_A_line_data_y, colors=['red'], default_size=200, 
                             marker='square')
vaccine_B_scatter = plt.scatter(vaccine_B_line_data_x, vaccine_B_line_data_y, colors=['blue'], default_size=200, 
                             marker='triangle-up')

update_totals(grid)
update_probabilities(grid)
update_plot(None, grid, vaccine_A_line, vaccine_B_line, vaccine_A_line_total, vaccine_B_line_total)

plt.xlabel('Number Treated')
plt.ylabel('Cured')

for i in [2,3]:
    for j in [1,2,3,4,5,6]:
        grid[i, j].observe(lambda change: update_plot(change, grid, vaccine_A_line, vaccine_B_line, vaccine_A_line_total, vaccine_B_line_total))
        grid[i, j].observe(lambda change: on_text_update(change, grid))

grid.layout.width='800px'
grid.layout.height='300px'

vaccine_grid = widgets.VBox([grid, fig_vaccines])

#drug_grid.layout.align_items = 'flex-start'
vaccine_grid.layout.align_items = 'center'
vaccine_grid.layout.align_content = 'stretch'
vaccine_grid.layout.width = '800px'
vaccine_grid.layout.height = '600px'
vaccine_grid.layout.border = 'black solid 1px'

display(vaccine_grid)

Some potentially helpful details:

Notice the probabilities reported below the data table.
For example, under the "Males" column, "Pr(Cure | A) = 2/6" means "For a male treated with vaccine A, the probability of being cured was 2/6".

If, for example, vaccine A is more successful than vaccine B in treating males, then that probability will be colored red.

The newspaper reports that vaccine A is better for males and females, but vaccine B is better overall.
This means you want the "A" probabilities to be colored red for males and females, but the "B" probability to be colore for the total.

## CHALLENGE QUESTIONS

<div class="alert alert-success" role="alert">
</div>

- Which of our three examples does this remind you of?
(batting averages)
- What lessons from that example would you share with your friend? 
(Leading (greater success) in each year (gender) does not mean leading overall. ; 
Simpson's Paradox relies requires an imbalance in at-bats (number treated) - and we see that is the case here.)
- How does the slope of each line segment relate to probability?
(number cured / number treated = probability of being cured if you take that drug)
- Based on the information in the news reports, which vaccine would you choose?
(If you are male, vaccine A is the best choice. If you are female, vaccine A is the best choice.)
- Imagine that only one vaccine can be brought to production. Which one would you choose?
(Provide this answer:
This is tricky! 
Assume that the goal is to cure the greatest number of people, irrespective of gender.
If there are $M$ males in the population and $F$ females, then you would compute:
$$ M * Pr(\textrm{Cured} | \textrm{vaccine A for males}) + F * Pr(\textrm{Cured} | \textrm{vaccine A for females})$$
and compare it with 
$$ M * Pr(\textrm{Cured} | \textrm{vaccine B for males}) + F * Pr(\textrm{Cured} | \textrm{vaccine B for females})$$
)

## Wrap up

<hr style="height:6px">

In this lesson we've seen three different flavors of Simpson's Paradox.

This is a difficult concept, so if you've made it this far, give yourself a high-five!

Hopefully your "Simpson Radar" is now improved, and you will be better able to spot these features in real-world data.

## Congratulations!

<hr style="height:6px">
<hr style="height:6px">
<hr style="height:6px">

### Brilliant Teaching Principles (https://brilliant.org/principles/):

<hr style="height:6px">

Highlight the ways in which this lesson attends to the  Brilliant Teaching Principles.

1. Excites: The greatest challenges to education are disinterest and apathy.

Fish are colorful and silly looking cartoons.
I employ variety of learning modes - prose, numbers, algebra, graphics.
I present challenges as games.

2. Cultivates curiosity: Questions and storytelling that cultivate natural curiosity are better than the threat of a test.

There are no threats intended here.
I incorporate a range of difficulty including some very beginner.
Also, the interactive "games" have no time limit - this deemphasizes success vs failure, certainly in a test sense.
    
3. Is active: Effective learning is active, not passive. Watching a video is not enough.

Lots of engagement, especially in the interactive games.
Questions interspersed maintain thoughtfulness.
Baseball is physical - it gets people thinking about motion and action.

4. Is applicable: Use it or lose it: it is essential to apply what you're learning as you learn it.

Drug example is very applicable.
Making decisions (or even just understanding your doctor) in the face of statistics is a valuable skill.

5. Is community driven: A community that challenges and inspires you is invaluable.

Not sure how to incorporate this.

6. Doesn't discriminate: Your age, country, and gender don't determine what you are capable of learning.

I took care to choose examples that seemed accessible to a wide audience.
Fish are innocuous.
Simple shapes and colors are accessible across grades, languages, cultures.
Baseball is pretty accessible cross-culture/class.

7. Allows for failure: The best learners allow themselves to make many mistakes along their journey.

Interactive games offer a feedback for learning through a process of incremental successes and "failures".

8. Sparks questions: The culmination of a great education isn't knowing all the answers — it's knowing what to ask.

I hope the conversational, inquiry-based style along with the non-judgemental tone encourages questions.